# CMPINF Final Project
## Evan Tall, Jerry Huang, Hailey Trampel

## How we ranked 'bestness':
* How **safe** the neighborhood is
* How many **hospitals** the neighborhood has
* How many **bridges** the neighborhood has 



In [66]:
import pandas as pd
import numpy as np
import ssl

## Analyzing how safe the neighborhoods are
First, we imported the data for Pittsburgh Area arrests, as reported by the Pittsburgh Police.

In [67]:
pgh_crime_data = pd.read_csv("crime_data.csv")
crime_data = pgh_crime_data["INCIDENTNEIGHBORHOOD"].value_counts().sort_values(ascending=True)
crime_dict = {'Arrest Reports:': crime_data}
crime = pd.DataFrame(crime_dict)
crime.index.name = "Neighborhood"
crime.sort_index(axis = 0)

#i want to name the index 'Neighborhood' so that when I combine the data frames, the indexes are all named the
#same thing and can be merged.

,Arrest Reports:
Neighborhood,
Allegheny Center,651
Allegheny West,75
Allentown,539
Arlington,168
Arlington Heights,100
Banksville,141
Bedford Dwellings,336
Beechview,686
Beltzhoover,313


In [4]:
crime_bad = pgh_crime_data["INCIDENTNEIGHBORHOOD"].value_counts()

In [68]:
public_safety_data = pd.read_csv('public_safety_data.csv')
safety=public_safety_data.sort_values(by = 'Part 1 Crime per 100 Persons (2010)', ascending=True)
safety_dict = safety[['Neighborhood','Part 1 Crime per 100 Persons (2010)']]
safety_df = pd.DataFrame(safety_dict)
safety_df.set_index('Neighborhood').sort_index(axis = 0)

,Part 1 Crime per 100 Persons (2010)
Neighborhood,
Allegheny Center,7.6
Allegheny West,8.2
Allentown,6.4
Arlington,2.7
Arlington Heights,7.8
Banksville,1.5
Bedford Dwellings,3.5
Beechview,2.4
Beltzhoover,4.3


## Counting the number of bridges per region

In [69]:
pgh_bridge_data = pd.read_csv("bridge_data.csv")
pgh_bridge_data["neighborhood"].value_counts()
bridge_data = pgh_bridge_data["neighborhood"].value_counts()
bridge_dict = {'Number of Bridges:': bridge_data}
bridges = pd.DataFrame(bridge_dict)
bridges.index.name = "Neighborhood"
bridges.sort_index(axis = 0) 
#i want to name the index 'Neighborhood' so that when I combine the data frames, the indexes are all named the
#same thing and can be merged.

,Number of Bridges:
Neighborhood,
Allegheny Center,2
Allegheny West,3
Allentown,1
Arlington,2
Banksville,1
Beechview,1
Bloomfield,1
Bluff,2
Bon Air,3


## Calculating the Best Using our Data

In [77]:
pd.concat([crime, safety_df, bridges], axis=1, sort=True)

/ihome/crc/install/python/miniconda3-3.7/lib/python3.7/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
/ihome/crc/install/python/miniconda3-3.7/lib/python3.7/site-packages/pandas/core/indexes/api.py:69: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)


,Arrest Reports:,Neighborhood,Part 1 Crime per 100 Persons (2010),Number of Bridges:
Mt. Oliver Neighborhood,2.0,NaN,NaN,NaN
Troy Hill-Herrs Island,6.0,NaN,NaN,NaN
Mt. Oliver Boro,12.0,NaN,NaN,NaN
Ridgemont,13.0,NaN,NaN,NaN
Regent Square,19.0,NaN,NaN,1.0
Swisshelm Park,20.0,NaN,NaN,1.0
New Homestead,22.0,NaN,NaN,NaN
Central Northside,23.0,NaN,NaN,1.0
Chartiers City,23.0,NaN,NaN,NaN
East Carnegie,27.0,NaN,NaN,NaN


In [76]:
first = pd.concat([crime, bridges], axis=1, sort=False)
first

,Arrest Reports:,Number of Bridges:
Mt. Oliver Neighborhood,2,NaN
Troy Hill-Herrs Island,6,NaN
Mt. Oliver Boro,12,NaN
Ridgemont,13,NaN
Regent Square,19,1.0
Swisshelm Park,20,1.0
New Homestead,22,NaN
Central Northside,23,1.0
Chartiers City,23,NaN
East Carnegie,27,NaN


## The Best Neighborhood in Pittsburgh is... OAKWOOD
### Oakwood ranked:
* 6th for least arrests per year
* 14th for best public safety, based off of crime reports per person
* 4th for most bridges
  
This neighborhood was the only one that was in the top 20 of each of our categories. We decided 